In [14]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import hazm

In [3]:

def create_output(output_type, marker, span):
    return {"type":output_type, "marker":marker, "span":repr(span)}

In [46]:
import re


class Symbolizer:
    def __init__(self):
        self.symbols = []
        with open('namad.txt') as symbols_file:
            line = symbols_file.readline()
            while line:
                n = line.split('\t')
                self.symbols.append(n[0])
                # TODO: we can add more information to a symbol here
                # namad_description.append(n[1])
                line = symbols_file.readline()

    def symbolize(self, sentence):
        output = []
        print(output)
        for namad in self.symbols:
            for m in re.finditer(namad, sentence):
                print('---------', namad, len(self.symbols))
                output.append(create_output("نماد", namad, m.span()))
                print(output)
        return output

In [6]:
events = ['مثبت','سود','صعود']


In [19]:
def event_extractor(noun_phrase):
    output=[]
    for event in events:
        for m in re.finditer(event, noun_phrase.group()):
                output.append(create_output("واقعه", noun_phrase.group()[1:-3], noun_phrase.span()))
    return output


In [47]:
sentence = 'رشد قیمت‌ها باعث ایجاد صف خرید در سهم پرشیا شد'
symbolizer = Symbolizer()
output = symbolizer.symbolize(sentence)
print(output)

# tagger = hazm.POSTagger(model='../resources/postagger.model')
# chunker = hazm.Chunker(model='../resources/chunker.model')
# tagged = tagger.tag(hazm.word_tokenize(sentence))
# # sentence_tokens = word_tokenize(sentence)
# # print(chunker.parse(tagged))b.group()
# print(tagged)
# a = hazm.tree2brackets(chunker.parse(tagged))
# print(a)
# # for b in  re.finditer(r"\[[آ-یء چ]+NP\]", a):
# for b in  re.finditer(r"\[[^\]]*NP\]", a):
#     print('--------------')
#     print(b.group())
#     print(event_extractor(b))


AttributeError: 'Symbolizer' object has no attribute 'symbols'

In [56]:
import hazm
from __future__ import unicode_literals
tagger = hazm.POSTagger(model='../resources/postagger.model')
print(tagger.tag(hazm.word_tokenize('ما بسیار کتاب می‌خوانیم'))[3][0])
# tagger.tag(hazm.word_tokenize('ما بسیار کتاب می‌خوانیم'))
tagger.tag(hazm.word_tokenize('کتاب خواندن را دوست داریم'))
tagger.tag(hazm.word_tokenize('من درخت بلند را دیدم'))
tagger.tag(hazm.word_tokenize('مرد زبان نفهم'))
tagger.tag(hazm.word_tokenize('مرد سود ده'))

می‌خوانیم


[('مرد', 'Ne'), ('سود', 'Ne'), ('ده', 'NUM')]

In [93]:
word = 'کتاب خواندن را دوست داریم'
word = '۵ گل زیبا خریدم'
word = 'سهام ۳ واحد افزایش یافت'
word = 'هوا ۵ درجه گرم شد'
chunker = hazm.Chunker(model='resources/chunker.model')
chunker = hazm.RuleBasedChunker()
tagged = tagger.tag(hazm.word_tokenize(word))
print(chunker.parse(tagged))
print(hazm.tree2brackets(chunker.parse(tagged)))
chunker.parse(tagged).draw()

(S (NP هوا/N) (NP ۵/NUM درجه/N) (ADJP گرم/AJ) (VP شد/V))


In [ ]:
import nltk

nltk.ChunkParserI('a big cat was dead')

In [40]:
lst = [u'\u200c', u'\u5de5', 'ب\u200cر', 'امی‌ن']
print(lst)
print('امی\u200cن')
print('امی‌ن')

['\u200c', '工', 'ب\u200cر', 'امی\u200cن']
امی‌ن
امی‌ن


In [55]:
import nltk
nltk.download('tagsets')
nltk.help.upenn_tagset()

[nltk_data] Downloading package tagsets to /home/amin/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai

In [85]:
# nltk.download('punkt')
# nltk.download('popular')
# nltk.download('universal_tagset')
# text = nltk.word_tokenize("He is a good, tall boy")
text = nltk.word_tokenize("He is a little tall boy")
# text = nltk.word_tokenize("the little crazy cat sat on the mat")
# text = nltk.word_tokenize("amin is clever and good boy")
# text = nltk.word_tokenize("And now for something completely different")
nltk.pos_tag(text, tagset='universal')
# nltk.pos_tag(text)

[('He', 'PRON'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('little', 'ADJ'),
 ('tall', 'ADJ'),
 ('boy', 'NOUN')]

In [78]:
unchunked_sent = 'the little cat sat on the mat'
rule1 = nltk.chunk.regexp.ChunkRule('<NN|DT>+',
    'Chunk sequences of NN and DT')
chunkparser = nltk.chunk.regexp.RegexpParser( [rule1] )
chunkparser.parse(unchunked_sent)

TypeError: Expected string or list of RegexpChunkParsers for the grammar.

In [91]:
import nltk
text = "learn php from guru99"
text = 'the little crazy cat is sitting on the mat'
tokens = nltk.word_tokenize(text)
print(tokens)
tag = nltk.pos_tag(tokens)
print(tag)
grammar = """
NP: {<DT>?<JJ>*<NN>} 
VP: {<VB.?>*}
"""
cp  = nltk.RegexpParser(grammar)
result = cp.parse(tag)
print(result)
result.draw()    # It will draw the pattern graphically which can be seen in Noun Phrase chunking 

['the', 'little', 'crazy', 'cat', 'is', 'sitting', 'on', 'the', 'mat']
[('the', 'DT'), ('little', 'JJ'), ('crazy', 'JJ'), ('cat', 'NN'), ('is', 'VBZ'), ('sitting', 'VBG'), ('on', 'IN'), ('the', 'DT'), ('mat', 'NN')]
(S
  (NP the/DT little/JJ crazy/JJ cat/NN)
  (VP is/VBZ sitting/VBG)
  on/IN
  (NP the/DT mat/NN))
